In [3]:
# --- Ingestion: Load and chunk your 4 KMS PDFs ---

import fitz  # PyMuPDF
from uuid import uuid4
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss



/opt/anaconda3/envs/dsenv/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
# Function to extract text from multiple PDFs
def extract_text_from_pdfs(pdf_paths):
    """Extract text from multiple PDF files using PyMuPDF (fitz)."""
    all_text = ""
    for pdf_path in pdf_paths:
        try:
            doc = fitz.open(pdf_path)
            all_text += "\n".join([page.get_text() for page in doc])
            doc.close()
        except Exception as e:
            print(f"Error extracting text from {pdf_path}: {e}")
    return all_text



In [5]:
# Paths to your PDFs (make sure commas separate items!)
pdf_files = [
    "contents/nist_80053r5.pdf",
]



In [7]:
# Extract text
pdf_text = extract_text_from_pdfs(pdf_files)
print("Sample text:\n", pdf_text[:500])  # sanity check

# Split into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n", " ", ""]
)
chunks = splitter.split_text(pdf_text)
print("Total chunks:", len(chunks))

# Wrap chunks into Document objects
documents = [Document(page_content=chunk, metadata={"source": "KMS_PDFs"}) for chunk in chunks]
uuids = [str(uuid4()) for _ in range(len(documents))]



Error extracting text from contents/nist_80053r5.pdf: module 'fitz' has no attribute 'open'
Sample text:
 
Total chunks: 0


In [ ]:
# Initialize embeddings model (Google Generative AI example)
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
print("✅ Embeddings model initialized.")

/var/folders/12/sxr7_b5d455gvm_89tb6162c0000gn/T/ipykernel_9962/382558720.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# Build FAISS vector store
dim = len(embeddings.embed_query("test"))  # get embedding dimension
index = faiss.IndexFlatL2(dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(documents=documents, ids=uuids)
vector_store.save_local("faiss_index_kms")

print("✅ Vector store built and saved from KMS PDFs.")

NameError: name 'embeddings' is not defined